# Prep Data

In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  
import contractions

In [2]:
%cd datasets

c:\Users\willc\Documents\wills-ensemble\datasets


In [3]:
with open('twitter.txt', 'r') as f:
    lines = f.readlines()

lines

['693136527816331264\ta tiny alpine town in northern italy just welcomed its first baby in 28 years 🍼👶 URL URL\tnon-rumor\n',
 '755475529294352385\tohio police officer hospitalized after eating shards of glass in sandwich from columbus restaurant blackliesmatter URL\tunverified\n',
 '744390771869102080\tone of the men who wanted to copy the #orlando shooter was stopped in l.a. he was a liberal democrat and #berniebro. URL\tunverified\n',
 '534445263528947712\tmissouri governor declares state of emergency ahead of grand jury URL\tfalse\n',
 '691027026552229888\tdonald trump spokesperson decried lack of "pure breeds" running for president URL URL\tnon-rumor\n',
 '514540926640087040\tthe iphone 6 plus really does have a bending problem URL URL\tfalse\n',
 '407161635439013888\tpaul walker dies, millions cried. millions died of human hunger, no one simply gives a fk.\ttrue\n',
 '731166399389962242\t🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL\tunverified\n',


In [19]:
twitter_df = pd.DataFrame(columns=["text", "verdict"])
for line in lines:
    sections = line.split('\t')
    verdict = sections[2][:-1]
    if verdict == "non-rumor":
        continue
    if verdict == "unverified":
        verdict = "false"

    data = [sections[1], verdict]
    twitter_df.loc[len(twitter_df)] = data

,text,verdict
0,ohio police officer hospitalized after eating ...,false
1,one of the men who wanted to copy the #orlando...,false
2,missouri governor declares state of emergency ...,false
3,the iphone 6 plus really does have a bending p...,false
4,"paul walker dies, millions cried. millions die...",true
...,...,...
1724,kim davis angers 'eye of the tiger' creator af...,false
1725,ever wonder why the most mass shootings ever h...,false
1726,"#breaking hostages held inside sydney cafe, is...",true
1727,newsflash⚡️the #orlando shooter was a muslim a...,false


In [24]:
for index, row in twitter_df.iterrows():
    print(row.text)

ohio police officer hospitalized after eating shards of glass in sandwich from columbus restaurant blackliesmatter URL
one of the men who wanted to copy the #orlando shooter was stopped in l.a. he was a liberal democrat and #berniebro. URL
missouri governor declares state of emergency ahead of grand jury URL
the iphone 6 plus really does have a bending problem URL URL
paul walker dies, millions cried. millions died of human hunger, no one simply gives a fk.
🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL
#mikebrown wanted to walk down the street. and he is dead. and the #ferguson pd is using his silence in death to make him a criminal.
19-year-old girl wakes up during brain surgery, asks doctors ‘how’s it going?’ URL via @foxnewshealth
will you be able to float on jan. 4, 2015, aka zero g day? hint: no. URL URL
great @banksy homage to charlie hebdo victims: rt @kowalshki: nice work from banksy #jesuischarlie URL
man cut off penis and threw it in bin after g

In [25]:
# Clean text

def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]', ' ', text)

def remove_url(text):
    return re.sub('URL', '', text)

def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def clean_text(text):
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    #text = remove_stopwords(text)
    #text = stemming_words(text)
    #text = lemmatize_words(text)
    return text
    
# apply
twitter_df['text'] = twitter_df['text'].apply(clean_text)
twitter_df["text"].values

array(['ohio police officer hospitalized after eating shards of glass in sandwich from columbus restaurant blackliesmatter',
       'one of the men who wanted to copy the  orlando shooter was stopped in l a  he was a liberal democrat and  berniebro ',
       'missouri governor declares state of emergency ahead of grand jury',
       ...,
       ' breaking hostages held inside sydney cafe  islamic flag held up',
       'newsflash  the  orlando shooter was a muslim and a democrat   not a republican or christian   gaypride    ',
       'great scott   backtothefuture    almost  nailed its      predictions   hplabs makes theirs '],
      dtype=object)

In [26]:
print(twitter_df.shape)
twitter_df.to_csv("twitter.csv")

(1729, 2)
